In [11]:
import os
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
from data_tools import preprocess_data
from part2 import config
from part2.model import Classifier

In [2]:
# 设置环境变量以解决OpenMP冲突
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
# 加载数据
train_file = config.TRAIN_FILE
test_file = config.TEST_FILE
X_train, y_train, X_test, y_test, tokenizer = preprocess_data(train_file, test_file)

In [4]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((161297, 170), (161297, 3), (53766, 170), (53766, 3))

In [5]:
y_train,X_train

(array([[0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        ...,
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.]], dtype=float32),
 array([[   0,    0,    0, ...,    2, 3806, 1551],
        [   0,    0,    0, ...,    3,  212,  312],
        [   0,    0,    0, ...,   24, 9919, 8144],
        ...,
        [   0,    0,    0, ...,    4,  504,   81],
        [   0,    0,    0, ..., 1350,    2,  246],
        [   0,    0,    0, ...,  442,    9,   17]]))

In [7]:
X_train_tensor = torch.tensor(X_train, dtype=torch.long)
y_train_tensor = torch.tensor(y_train.argmax(axis=1), dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.argmax(axis=1), dtype=torch.long)

In [8]:
y_train_tensor[0:10],y_test_tensor[0:10]

(tensor([1, 1, 0, 1, 1, 2, 2, 1, 2, 1]),
 tensor([1, 1, 1, 1, 1, 2, 0, 1, 1, 2]))

In [9]:
# 创建DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [12]:
# 创建模型
model = Classifier()

In [13]:
# 损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 适用于多分类问题
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [14]:
# 训练模型时的其他函数
def train_model(model, train_loader, criterion, optimizer, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            # 清零梯度
            optimizer.zero_grad()

            # 前向传播
            outputs = model(inputs)
            # print(outputs)
            # print(labels)
            loss = criterion(outputs, labels)

            # 反向传播
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # 计算准确率
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(
            f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}, Accuracy: {correct / total}")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy}")
    return accuracy

In [15]:
# 训练模型
train_model(model, train_loader, criterion, optimizer, num_epochs=3)

100%|██████████| 1261/1261 [01:13<00:00, 17.11it/s]


Epoch [1/3], Loss: 0.6835809076333406, Accuracy: 0.7324624760534914


100%|██████████| 1261/1261 [01:18<00:00, 16.00it/s]


Epoch [2/3], Loss: 0.5356694067120079, Accuracy: 0.8023335833896477


100%|██████████| 1261/1261 [01:15<00:00, 16.64it/s]

Epoch [3/3], Loss: 0.47469321096539024, Accuracy: 0.8255578219061731


In [16]:
evaluate_model(model, test_loader)

100%|██████████| 421/421 [00:05<00:00, 76.59it/s]

Test Accuracy: 0.8178588699177919


0.8178588699177919